In [215]:
import requests
import json
import os
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

True

In [216]:
base_url = "https://jira.eficode.com"
access_token = os.getenv('JIRA_API_TOKEN')


In [217]:
url = f'{base_url}/rest/tempo-planning/1/allocation'
header = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}
res = requests.get(url=url, headers=header)

print(res.status_code)
data = res.json()

data

# data.('data.csv', index=False)


200


[{'id': 607,
  'assignee': {'key': 'rasmus.paltschik',
   'type': 'user',
   'userKey': 'JIRAUSER15067'},
  'planItem': {'id': 226025, 'type': 'ISSUE', 'projectId': 0},
  'scope': {'id': 226025, 'type': 'none'},
  'commitment': 60.0,
  'secondsPerDay': 16200,
  'includeNonWorkingDays': False,
  'start': '2023-04-19',
  'startTime': '12:53',
  'end': '2023-05-31',
  'description': '',
  'seconds': 502200,
  'created': '2023-04-19',
  'createdBy': 'rasmus.paltschik',
  'createdByKey': 'JIRAUSER15067',
  'updated': '2023-04-19',
  'updatedBy': 'rasmus.paltschik',
  'recurrence': {'endDate': '2023-05-31', 'rule': 'NEVER'}},
 {'id': 541,
  'assignee': {'key': 'juha.patrikainen',
   'type': 'user',
   'userKey': 'jpatrikainen'},
  'planItem': {'id': 227317, 'type': 'ISSUE', 'projectId': 0},
  'scope': {'id': 227317, 'type': 'none'},
  'commitment': 80.0,
  'secondsPerDay': 21600,
  'includeNonWorkingDays': False,
  'start': '2023-01-01',
  'startTime': '12:27',
  'end': '2023-06-30',
  'desc

In [218]:
df = pd.DataFrame(columns=['plan_id', 'employee', 'start', 'end', 'hours_day', 'employee_day'])


for plan in data:
    row = {'plan_id': plan['id'], 'employee': plan['assignee']['key'], 'start': plan['start'], 'end': plan['end'], 'hours_day': int(plan['secondsPerDay'])/(60*60)}
    df.loc[len(df)] = row

df['start'] = pd.to_datetime(df['start'], format='%Y/%m/%d')
df['end'] = pd.to_datetime(df['end'], format='%Y/%m/%d')

# Drop user '1'
value_to_drop = '1'
df = df[df['employee'] != value_to_drop]
# dfgroup = df

# df

In [219]:
""" def generate_date_range(row):
    return pd.date_range(start=row['start'], end=row['end'], freq='D')
# apply the function to each row, resulting in a new column with the date range as a pandas.date_range object
df['date_range'] = df.apply(generate_date_range, axis=1) """

" def generate_date_range(row):\n    return pd.date_range(start=row['start'], end=row['end'], freq='D')\n# apply the function to each row, resulting in a new column with the date range as a pandas.date_range object\ndf['date_range'] = df.apply(generate_date_range, axis=1) "

In [220]:
""" df['date_range'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('date_range')

df_summed = df_exploded.groupby(['date_range', 'employee'])['hours_day'].sum().reset_index()
# df_summed
"""

""" df['date_range'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('date_range')

df_summed = df_exploded.groupby(['date_range', 'employee'])['hours_day'].sum().reset_index()
df_summed """






df['Date'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('Date')

# plans = df_exploded.groupby(['date', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()

# plans[plans['employee'] == 'tuomas.kara']

df_hours_sum = df_exploded.groupby(['Date', 'employee'])['hours_day'].sum().reset_index()
df_plan_agg = df_exploded.groupby(['Date', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()

merged_data = pd.merge(df_hours_sum, df_plan_agg, on=['Date', 'employee'])

df_summed = df_exploded.groupby(['Date', 'employee'])['hours_day'].sum().reset_index()
plans = df_exploded.groupby(['Date', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()
# Merge the new data frame with the employee data
merged_data = pd.merge(df_summed, plans, how='left')


# Read in the date table
date_table = pd.read_csv('Datetable.csv', parse_dates=['Date'])
# Convert the date column to a datetime object
date_table['Date'] = pd.to_datetime(date_table['Date'], format='%Y/%m/%d')

# date_table.set_index('Date', inplace=True)


df['Date'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)


df_exploded = df.explode('Date')

df_hours_sum = df_exploded.groupby(['Date', 'employee'])['hours_day'].sum().reset_index()
df_plan_agg = df_exploded.groupby(['Date', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()

merged_data = pd.merge(df_hours_sum, df_plan_agg, on=['Date', 'employee'])


In [221]:

begin = '2023/01/01'
end = '2023/12/31'

all_dates = pd.date_range(start=begin, end=end, freq='D')
# all_dates = pd.Date(start=begin, end=end, freq='D')
all_names = merged_data['employee'].unique()
name_date_df = pd.MultiIndex.from_product([all_names, all_dates], names=['employee', 'Date']).to_frame(index=False)

df_all_days = pd.merge(name_date_df, merged_data, on=['employee', 'Date'], how='left').fillna({'hours_day': 0})

df_all_days['Date'] = df_all_days['Date'].dt.floor('D')
# df_all_days[df_all_days['employee'] == 'tuomas.kara']





merged_data = df_all_days.copy()

# merged_data.set_index('Date', inplace=True)

merged_data['allocation'] = 'Not fully allocated'

# merged_data['is_workday'] = date_table['is_workday']
merged_data = merged_data.merge(date_table[['Date', 'is_workday']])



merged_data.loc[merged_data['hours_day'] >= 7.5, 'allocation'] = 'Fully allocated'
merged_data.loc[merged_data['hours_day'] <= 0.0, 'allocation'] = 'Not allocated'

merged_data.loc[merged_data['is_workday'] == False, 'allocation'] = 'Not working day'

merged_data.loc[merged_data['is_workday'] == False, 'hours_day'] = 0


merged_data = merged_data.merge(date_table[['Date', 'Year', 'Month', 'Week', 'Day']])

""" merged_data = merged_data.merge(date_table[['Date', 'Year']])
merged_data = merged_data.merge(date_table[['Date', 'Month']])
merged_data = merged_data.merge(date_table[['Date', 'Day']])
merged_data = merged_data.merge(date_table[['Date', 'Week']])
 """

" merged_data = merged_data.merge(date_table[['Date', 'Year']])\nmerged_data = merged_data.merge(date_table[['Date', 'Month']])\nmerged_data = merged_data.merge(date_table[['Date', 'Day']])\nmerged_data = merged_data.merge(date_table[['Date', 'Week']])\n "

In [222]:
merged_data.to_csv('updated_employee_data.csv', index=False)
